## Inter-subject analysis

This notebook will 
- Process and generate reports for all the dataset following procedure detailed in Bruxism detection
- Display several metrics for all patients of the datasets
- Display group analysis differences between tinnitus overnight increase patients and stable overnight patients (for tinnitus masking volume and VAS scales)
- Display correlation plots between tinnitus evolution overnight and number of bruxisme episodes per hour (for tinnitus masking volume and VAS scales)

In [1]:
import os
PATH = os.getcwd() 
import sys
sys.path.append(PATH + '/../')
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
from statannot import add_stat_annotation
from itertools import combinations
from tinnsleep.data import read_etiology_file

print("config loaded")


config loaded


In [2]:
#Setting parameters
results_file = "data/reports_and_datas_bruxism.pk"
info_file = "data/data_info.csv"
etiology_file = "data/etiology.xlsx"

# PART 1 Compute Results
Skip to Part 2 if results files has been computed already

## Processing of the dataset and report generation

In [ ]:
# change to --overwrite True if you want to force to recompute the results
%run ../compute_results.py --bruxism True --mema False --overwrite False 


config loaded.
Performs Bruxism: <True>
Performs MEMA: <False>
Will overwrite existing results: <False>
parameters set


# Part 2 Load and Analysis Results
Part 1 is optional if the .pk has been saved

### Loading results

In [ ]:
results = pd.read_pickle(results_file).to_dict()
THR_classif = np.array(results[list(results.keys())[0]]["THR_classif"])

data_info = pd.read_csv(info_file, sep=",").query('emg == 1 & included == 1 & session == "1"')
etiology = read_etiology_file(etiology_file)
data_info = data_info.merge(etiology,on="subject")

print(f"Valid files for bruxism {data_info.shape[0]}")
print(data_info[["filename", "condition", "hyperacusis"]])

# keep only results from query
results= {filename:results[filename] for filename in data_info["filename"].values}

## 2.a Number of episode per hour

In [ ]:
# vizualization function
def plot_inter_subjects(results, value): 
    for filename in results.keys():
        y_axis=[]
        for report in results[filename]["reports"]:
            y_axis.append(report[value])
        plt.plot(THR_classif[:,1], y_axis, label=filename)
    plt.title('Bruxism scoring')
    plt.xlabel('Threshold for classification')
    plt.ylabel(value)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left', ncol=1)

In [ ]:
plt.close("all")
plt.figure()
value = "Number of episodes per hour"
plot_inter_subjects(results, value)

## 2.b Burst Duration 

In [ ]:
plt.figure()
value = "Total burst duration"
plot_inter_subjects(results, value)

## 2.c Logs of the preprocessing

In [ ]:
file=[]
suppr_imp=[]
suppr_amp=[]
suppr_OMA=[]
for elm in results.keys():
    #print(results[elm]["log"])
    #if results[elm]["log"]["bruxism"]['suppressed_overall']/results[elm]["log"]["bruxism"]['total_nb_epochs'] > 0.0001:
        sum_suppr= results[elm]["log"]["IMP"]['suppressed_overall']/results[elm]["log"]["IMP"]['total_nb_epochs'] + results[elm]["log"]["bruxism"]['suppressed_overall']/results[elm]["log"]["bruxism"]['total_nb_epochs'] + results[elm]["log"]["OMA"]['suppressed_overall']/results[elm]["log"]["OMA"]['total_nb_epochs']
        file.append(elm.split(os.path.sep)[-1][:-4] + " | " + 
                    str("%.1f" % (100.0 *sum_suppr))
                   + " % tot_suppr")
        suppr_imp.append(100*(results[elm]["log"]["IMP"]['suppressed_overall']/results[elm]["log"]["IMP"]['total_nb_epochs']))
        suppr_amp.append(100*results[elm]["log"]["bruxism"]['suppressed_overall']/results[elm]["log"]["bruxism"]['total_nb_epochs'])
        suppr_OMA.append(100*results[elm]["log"]["OMA"]['suppressed_overall']/results[elm]["log"]["OMA"]['total_nb_epochs'])
       


fig, ax = plt.subplots(figsize=(8, 13))
rect1 = ax.barh(file, suppr_imp)
rect2 = ax.barh(file, suppr_amp, left=suppr_imp)
starts=[]
for i in range(len(suppr_imp)):
    starts.append(suppr_imp[i]+ suppr_amp[i])
rect3 = ax.barh(file, suppr_OMA, left=starts)
plt.xlabel("% epochs rejected by imp_thr (bl) and by ampl_thr (or) and OMA (gr)")
plt.tight_layout()
plt.show()


## 2.d Comparing episode frequency with variability tinnitus

In [ ]:
# d.1 GROUP BY TINNITUS VOLUME VARIATION
reports = None
for filename in results.keys():
    report = pd.DataFrame(results[filename]["reports"])
    report["THR_classif"] = THR_classif[:,1]
    report["filename"] = filename
    if reports is None:
        reports = report
    else:
        reports = reports.append(report, ignore_index=True)

        
import math     
def map_category(x):
    if x < (-0.1):
        return "decrease"
    elif x > (0.1):
        return "increase"
    elif math.isnan(x):
        return "control"
    else:
        return "stable"
data_info["category"] = data_info["mask_delta"].apply(map_category)

reports = reports.merge(data_info, on="filename")

# d.2 reparing values for correlation scatter plots
    
print("Nb_patients in each group : ")
for category in list(data_info['category'].unique()):
    print(f"Tinnitus {category} : {(data_info['category']==category).sum()}, {(list(data_info[data_info['category']==category]['subject']))}")

report_mean = reports.groupby(['category', 'THR_classif']).mean()
report_std =  reports.groupby(['category', 'THR_classif']).std()

# d.3 Displaying
plt.figure()
value = "Number of episodes per hour"
for category in list(report_mean.index.get_level_values(0).unique()):
    plt.errorbar(THR_classif[:,1],report_mean.loc[category][value], report_std.loc[category][value], label = category)
plt.title("Influence of bruxism with patient category" )
plt.xlabel('Thresholding of classification')
plt.ylabel(value)
plt.legend()

## 2.e Etiology

In [ ]:
print(list(reports.keys()))

In [ ]:
# ETIOLOGIES
from tinnsleep.visualization import etiology_report_with_plot

data = reports
variable = "Number of episodes per hour"

hue = "THR_classif"
threshold = 4

etiologies = ['etiology', 'obstructed_ear', 'otalgy', 'hyperacusis', 'jaw_popping', 'jaw_pain_and_fatigue', 
            'somatosensory_modulation', 'nap_modulation', 'snoring']
for etiology in etiologies:
    etiology_report_with_plot(data, etiology, variable, hue=hue, hue_stats=threshold)

## 2.f Screen for correlated variables

In [ ]:
quantitative_variables = list(results[list(results.keys())[0]]["reports"][0].keys())[0:-1] #(remove power ratio)
print(f'variables: {quantitative_variables}')
effect_variable = ["mask_delta", "mask_per", "VAS_I_delta", "VAS_I_per", "VAS_L_delta", "VAS_L_per"]

In [ ]:
import seaborn as sns
from tinnsleep.visualization import regression_report_with_plot
plt.close("all")

data = reports.query("category != 'control' & THR_classif == 4.")
meta_results = regression_report_with_plot(data, effect_variable, quantitative_variables, conditions='THR_classif', title="Effect of Bruxism")

In [ ]:
meta_results.query("pvalue < 0.05")